In [ ]:
import oat_python as oat

import numpy as np
import plotly.graph_objects as go

# Generate a point cloud

In [ ]:
cloud               =   oat.point_cloud.spiral_sphere(npoints=300, noise_scale=0.07, random_seed=0)

#   PLOT THE POINT CLOUD
trace               =   go.Scatter3d(x=cloud[:,0],y=cloud[:,1],z=cloud[:,2], mode="markers", marker=dict(opacity=1, size=4, color=cloud[:,2], colorscale="Aggrnyl"))
fig                 =   go.Figure(data=trace)
fig.update_layout(title=dict(text="Shere"), height=800,width=850) 
fig.show()

# Compute persistent homology

We compute persistent homology by factoring the boundary matrix.  The following cell generates a sparse distance matrix and feeds it to the persistent homology solver.  The result is a factored boundary matrix.  We will extract information from this matrix in the following cells.

In [ ]:
dissimilairty_matrix    =   oat.dissimilarity.matrix_from_cloud(            
                                cloud                   =   cloud,
                                dissimilarity_max       =   0.5, # edges with length > 0.5 are excluded from the filtration
                            )

# build and factor the boundary matrix
factored                =   oat.rust.FactoredBoundaryMatrixVr( 
                                dissimilarity_matrix    =   dissimilairty_matrix,
                                homology_dimension_max  =   2,
                            )

Full details on `FactoredBoundaryMatrixVr` can be retreived with Python's `help` function.

In [ ]:
help(oat.rust.FactoredBoundaryMatrixVr)

# Plot the persistence diagram

In [ ]:
#   PLOT THE BARCODE

homology            =   factored.homology(
                            return_cycle_representatives     =   True,
                            return_bounding_chains          =   True,
                        )

fig_pd              =   oat.plot.pd( homology )
fig_pd.show()

# Plot the barcode

In [ ]:
fig_barcode              =   oat.plot.barcode( barcode=homology )
fig_barcode.show()

# Inspect homology and cycle representatives

The `homology` object is a data frame

In [ ]:
display(homology)

# Inspect a cycle representative and its bounding chain

By default, terms appear in reverse filtration order (ties are broken by reverse lexicographic order)

In [ ]:
homology["cycle representative"][420]

In [ ]:
homology["bounding chain"][420]

# Plot a representative

In [ ]:
#   FIND THE LONGEST BAR IN DIMENSION 1

def lifetime(p):
     """
     gets the lifetime of a feature; returns -infinity for features of dimension != 1
     """
     if homology["dimension"][p]!= 1:
          return -np.inf
     else:
          return homology["death"][p] - homology["birth"][p]

max( 
     range(homology.shape[0]),  # number of rows in the data frame
     key    =   lifetime,
)

In [ ]:
#   PLOT

edges               =   homology["cycle representative"][420]["simplex"].tolist() # the cycle
triangles           =   homology["bounding chain"][420]["simplex"].tolist() # the chain that bounds the cycle
coo                 =   cloud # coo stands for coordinate oracle

traces_edge         =   [ oat.plot.edge__trace3d( edge, coo  ) for edge in edges ]
trace_triangle      =   oat.plot.triangles__trace3d( triangles, coo ) 
trace_cloud         =   go.Scatter3d(x=cloud[:,0],y=cloud[:,1],z=cloud[:,2], mode="markers", marker = dict(opacity=0.5, size=3, color=cloud[:,1], colorscale="Aggrnyl"))

trace_cloud.update(showlegend=True, opacity=0.5, name="Point cloud")
trace_triangle.update(showlegend=True, legendgroup="triangles", opacity=0.5, name="Bounding chain", color="black")
for trace_number, trace in enumerate( traces_edge ):
    showlegend      =   trace_number == 0
    trace.update(showlegend=showlegend, legendgroup="edges", opacity=0.5, name="Initial cycle", line=dict(color="red", width=10))

fig = go.Figure(data= [trace_cloud] + traces_edge + [trace_triangle] )
fig.update_layout(title=dict(text="Cycle representative and bounding chain"))
fig.update_layout(height=800,width=850) 
fig.show()